In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [2]:
from pyquil.quil import Program
import pyquil.api as api
from pyquil.gates import *
qvm = api.QVMConnection()

In [3]:
from grove.pyvqe.vqe import VQE
from scipy.optimize import minimize

In [4]:
from pyquil.paulis import sX, sY, sZ, ID

We are running the same VQE gate combinations for LIH as before, this time using the CG minimizer.

In [5]:
hamiltonian = -0.096022*sZ(0) + -0.206128*sZ(0)*sZ(1) + 0.364746*sZ(1) + 0.096022*sZ(2) + -0.206128*sZ(2)*sZ(3) + -0.364746*sZ(3) + -0.145438*sZ(0)*sZ(2) + 0.056040*sZ(0)*sZ(2)*sZ(3) + 0.110811*sZ(0)*sZ(3) + -0.056040*sZ(0)*sZ(1)*sZ(2) + 0.080334*sZ(0)*sZ(1)*sZ(2)*sZ(3) + 0.063673*sZ(0)*sZ(1)*sZ(3) + 0.110811*sZ(1)*sZ(2) + -0.06373*sZ(1)*sZ(2)*sZ(3) + -0.095216*sZ(1)*sZ(3)
+ -0.012585*sX(0)*sZ(1) + 0.012585*sX(0) + 0.012585*sX(2)*sZ(3) + 0.012585*sX(2) + -0.002667*sX(0)*sZ(1)*sX(2)*sZ(3) + -0.002667*sX(0)*sZ(1)*sX(2) + 0.002667*sX(0)*sX(2)*sZ(3) + 0.002667*sX(0)*sX(2) + 0.007265*sX(0)*sZ(1)*sZ(3) + -0.007265*sX(0)*sZ(3) + 0.007265*sZ(1)*sX(2)*sZ(3) + 0.007265*sZ(1)*sX(2)
+ -0.029640*sX(0)*sX(1) + 0.002792*sX(1) + -0.029640*sX(2)*sX(3) + 0.002792*sX(3) + -0.008195*sX(0)*sX(2)*sX(3) + -0.001271*sX(0)*sX(3) + -0.008195*sX(0)*sX(1)*sX(2) + 0.028926*sX(0)*sX(1)*sX(2)*sX(3) + 0.007499*sX(0)*sX(1)*sX(3) + -0.001271*sX(1)*sX(2) + 0.007499*sX(1)*sX(2)*sX(3) + 0.009327*sX(1)*sX(3)
+ 0.029640*sY(0)*sY(1) + 0.029640*sY(2)*sY(3) + 0.028926*sY(0)*sY(1)*sY(2)*sY(3)
+ 0.002792*sZ(0)*sX(1) + -0.002792*sZ(2)*sX(3) + -0.016781*sZ(0)*sZ(2)*sX(3) + 0.016781*sZ(0)*sX(3) + -0.016781*sZ(0)*sX(1)*sZ(2) + -0.016781*sX(1)*sZ(2) + -0.009327*sZ(0)*sX(1)*sZ(2)*sX(3) + 0.009327*sZ(0)*sX(1)*sX(3) + -0.009327*sX(1)*sZ(2)*sX(3)
+ -0.011962*sZ(0)*sX(2)*sZ(3) + -0.011962*sZ(0)*sX(2) + 0.000247*sZ(0)*sZ(1)*sX(2)*sZ(3) + 0.000247*sZ(0)*sZ(1)*sX(2)
+ 0.039155*sZ(0)*sX(2)*sX(3) + -0.002895*sZ(0)*sZ(1)*sX(2)*sX(3) + -0.009769*sZ(0)*sZ(1)*sX(3) + -0.024280*sZ(1)*sX(2)*sX(3) + -0.008025*sZ(1)*sX(3)
+ -0.039155*sZ(0)*sY(2)*sY(3) + 0.002895*sZ(0)*sZ(1)*sY(2)*sY(3) + 0.024280*sZ(1)*sY(2)*sY(3)
+ -0.011962*sX(0)*sZ(1)*sZ(2) + 0.011962*sX(0)*sZ(2) + -0.000247*sX(0)*sZ(1)*sZ(2)*sZ(3) + 0.000247*sX(0)*sZ(2)*sZ(3)
+ 0.008195*sX(0)*sZ(1)*sX(2)*sX(3) + 0.0001271*sX(0)*sZ(1)*sX(3) + -0.007499*sX(0)*sX(1)*sZ(2)*sX(3)
+ -0.008195*sX(0)*sZ(1)*sY(2)*sY(3) + 0.008195*sX(0)*sY(2)*sY(3) + 0.007499*sY(0)*sY(1)*sZ(2)*sX(3)
+  -0.001271*sX(0)*sZ(1)*sZ(2)*sX(3) + 0.001271*sX(0)*sZ(2)*sX(3) + 0.008025*sZ(1)*sZ(2)*sX(3) + 0.009769*sZ(0)*sZ(1)*sZ(2)*sX(3)
+ -0.039155*sX(0)*sX(1)*sZ(2) + -0.002895*sX(0)*sX(1)*sZ(2)*sZ(3) + 0.024280*sX(0)*sX(1)*sZ(3) + -0.009769*sX(1)*sZ(2)*sZ(3) + 0.008025*sX(1)*sZ(3) + -0.001271*sZ(0)*sX(1)*sX(2)*sZ(3) + -0.001271*sZ(0)*sX(1)*sX(2) + 0.008025*sZ(0)*sX(1)*sZ(3)
+ 0.039155*sY(0)*sY(1)*sZ(2) + 0.002895*sY(0)*sY(1)*sZ(2)*sZ(3) + -0.024280*sY(0)*sY(1)*sZ(3) + 0.007499*sZ(0)*sX(1)*sX(2)*sX(3)
+ -0.008195*sX(0)*sX(1)*sX(2)*sZ(3) + -0.001271*sX(1)*sX(2)*sZ(3) + -0.007499*sZ(0)*sX(1)*sY(2)*sY(3)
+ 0.008195*sY(0)*sY(1)*sX(2)*sZ(3) + 0.008195*sY(0)*sY(1)*sX(2) + -0.009769*sZ(0)*sX(1)*sZ(2)*sZ(3)
+ -0.028926*sX(0)*sX(1)*sY(2)*sY(3) +  -0.007499*sX(1)*sY(2)*sY(3)  + -0.028926*sY(0)*sY(1)*sX(2)*sX(3) + -0.007499*sY(0)*sY(1)*sX(3)

In [6]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0), RY(params[1], 0), RZ(params[2],0),
                   RX(params[3], 1), RY(params[4], 1), RZ(params[5],1),
                   RX(params[6], 2), RY(params[7], 2), RZ(params[8],2),
                   RX(params[9], 3), RY(params[10], 3), RZ(params[11],3)
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([ -3.23741747e-05,  -3.23822102e-05,   9.99999549e-01,
         1.57075317e+00,   1.57083943e+00,   9.99999746e-01,
        -8.47096160e-08,  -1.08065279e-07,   9.99999747e-01,
        -2.97716039e-06,  -2.96565961e-06,   9.99999902e-01]), 'fun': -0.5494610002874158}


What happens if we try different number of variational parameters?
What if only rotation about X axis, only RX?

In [7]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1), 
                   RX(params[2], 2), 
                   RX(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14156030e+00,   3.14159262e+00,   6.37719107e-06,
        -1.30543459e-07]), 'fun': -0.8350149999763178}


A little bit farther from -0.835015. If we try this for RZ, we get:

In [8]:
def smallish_ansatz_HXYZ(params):
    return Program(RZ(params[0], 0),
                   RZ(params[1], 1), 
                   RZ(params[2], 2), 
                   RZ(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([ 1.,  1.,  1.,  1.]), 'fun': -0.3510110000000001}


Groundstate very closely represents some rotation about the x axis, involves very little rotation about the z axis. Let's try this for the T gate.

In [9]:
def smallish_ansatz_HXYZ(params):
    return Program(T(0),
                   T(1), 
                   T(2), 
                   T(3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([ 1.,  1.,  1.,  1.]), 'fun': -0.3510110000000001}


We see that we get the same (not great) result for T gates, since T gates are also just rotations about the Z axis, like RZ is.

In [10]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1), 
                   RX(params[2], 2), 
                   RZ(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14158409e+00,   3.14159456e+00,  -1.43381839e-05,
         1.00000023e+00]), 'fun': -0.8350149999931431}


In [11]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1), 
                   RZ(params[2], 2), 
                   RX(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14159611e+00,   3.14159260e+00,   9.99999369e-01,
        -3.82895099e-07]), 'fun': -0.8350149999997079}


In [12]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RZ(params[1], 1), 
                   RX(params[2], 2), 
                   RX(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  2.04413340e-06,   1.00000024e+00,   1.18568247e-06,
        -2.78572915e-06]), 'fun': -0.35101099999788166}


In [13]:
def smallish_ansatz_HXYZ(params):
    return Program(RZ(params[0], 0),
                   RX(params[1], 1), 
                   RX(params[2], 2), 
                   RX(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  9.99999988e-01,   3.14159613e+00,  -3.00263229e-06,
         1.48388105e-05]), 'fun': -0.7479109999547446}


We see that the X rotation for the second and first qubit (mostly the second qubit) matter much much more than the X rotation about the 3rd and 4th qubit. In fact, the scenario with an X rotation for the first three qubits, and a Z rotation for the last qubit yields the best possible result.

In [14]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1), 
                   RX(params[2], 2),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14158983e+00,   3.14159007e+00,  -4.64747236e-06]), 'fun': -0.8350149999979091}


In [15]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([-3.1415617 ,  3.14159059]), 'fun': -0.8350149999782054}


In [16]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0],0), RY(params[1], 0),
                   RX(params[2], 1), RY(params[3], 1), 
                   RX(params[4], 2), RY(params[5], 2),
                   RX(params[6], 3), RY(params[7], 3)
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([ -1.12886208e-05,  -1.13136441e-05,   1.57079366e+00,
         1.57079376e+00,   9.54649578e-07,   9.44639690e-07,
        -4.36669240e-06,  -4.37892158e-06]), 'fun': -0.5494609999812049}
